## Import Required packages

In [21]:
import os

from dotenv import load_dotenv
from openai import OpenAI, AssistantEventHandler
from typing_extensions import override

load_dotenv()

# Load the API key from the .env file
OPENAI_KEY = os.getenv("OPENAI_KEY")

 
client = OpenAI(api_key=OPENAI_KEY)

## Create Assistance

In [22]:
assistant = client.beta.assistants.create(
  name="Weather Agent BD",
  instructions="You are an expert in providing weather forecasts and climate data. Use your knowledge base to give accurate weather updates and predict future conditions for locations.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],  # You would need the appropriate weather tool here
)

## Create Vector Store

In [23]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Weather forcast")
# I want give the file path which is store in the same directory
file_paths = ["sorted_temp_and_rain_dataset_text_file.txt"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


## Update Assistance

In [24]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## Create a Tread

In [25]:
thread = client.beta.threads.create(
    messages=[
      {
          "role": "user",
          "content": "What was the average temperature in Dhaka in 2020?",
      }
    ]
)
print(thread.tool_resources.file_search)

None


## Create a run and check the output

In [26]:
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
        
    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
        
    
    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text

        print(message_content.value)
        
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()
    


assistant > file_search


assistant > The average monthly temperatures for Dhaka in 2020 were as follows:

- January: 16.56°C
- February: 18.85°C
- March: 24.56°C
- April: 27.41°C
- May: 27.99°C
- June: 28.03°C
- July: 28.39°C
- August: 28.51°C
- September: 28.28°C
- October: 27.61°C
- November: 22.97°C
- December: 17.91°C

To compute the annual average temperature, sum these monthly averages and divide by 12:

\[ \text{Annual Average Temperature} = \frac{16.56 + 18.85 + 24.56 + 27.41 + 27.99 + 28.03 + 28.39 + 28.51 + 28.28 + 27.61 + 22.97 + 17.91}{12} = \frac{297.07}{12} = 24.76 \text{°C} \]

So, the average temperature in Dhaka in 2020 was approximately 24.76°C【4:0†source】【4:1†source】.
